# Your objective
1. [Action Required] Read readme.md, run this notebook
2. Understand what the notebook mean
3. [Action Required] Try to train a lstm(hint: find the lstm in qlib/qlib/contrib/model/*)
4. backtest your lstm model. 
5. take a look at experiment/mlruns folder and take a look at ExpBacktest.ipynb. Understand what does keyword: "recorder", "experiment" means. 
6. [Action Required] Try to run backtest of your lstm result in ExpBacktest.ipynb file by loading the backtest data.


## Init Qlib

In [3]:
import sys
print(sys.path)
# get the parent directory of the current script
import os
cwd = os.path.abspath(os.path.join(os.getcwd(), "../"))
# insert cwd/+"../qlib" into sys.path as qlib
sys.path.insert(0, cwd + "/../qlib")
%load_ext autoreload
%autoreload 2
from pprint import pprint
from pathlib import Path
import pandas as pd
MARKET = "csi300"
BENCHMARK = "SH000300"
EXP_NAME = "ExpQuickStartGRUX"
from qlib.tests.data import GetData

GetData().qlib_data(exists_skip=True)
import qlib

print("qlib:",qlib.__path__)

qlib.init()
from qlib.data import D
p = Path(cwd+"/../data/qlib_data/cn_data/financial").expanduser()



['c:\\Users\\shiva\\Box/../qlib', 'c:\\Users\\shiva\\Box\\Capstone', 'c:\\Users\\shiva\\AppData\\Local\\Programs\\Python\\Python38\\python38.zip', 'c:\\Users\\shiva\\AppData\\Local\\Programs\\Python\\Python38\\DLLs', 'c:\\Users\\shiva\\AppData\\Local\\Programs\\Python\\Python38\\lib', 'c:\\Users\\shiva\\AppData\\Local\\Programs\\Python\\Python38', '', 'C:\\Users\\shiva\\AppData\\Roaming\\Python\\Python38\\site-packages', 'c:\\Users\\shiva\\AppData\\Local\\Programs\\Python\\Python38\\lib\\site-packages', 'c:\\Users\\shiva\\AppData\\Local\\Programs\\Python\\Python38\\lib\\site-packages\\win32', 'c:\\Users\\shiva\\AppData\\Local\\Programs\\Python\\Python38\\lib\\site-packages\\win32\\lib', 'c:\\Users\\shiva\\AppData\\Local\\Programs\\Python\\Python38\\lib\\site-packages\\Pythonwin']
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2023-12-15 14:42:45.615 | WARNING  | qlib.tests.data:qlib_data:175 - Data already exists: ~/.qlib/qlib_data/cn_data, the data download will be skipped
	If downloading is required: `exists_skip=False` or `change target_dir`
[24748:MainThread](2023-12-15 14:42:45,615) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[24748:MainThread](2023-12-15 14:42:45,615) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[24748:MainThread](2023-12-15 14:42:45,626) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/shiva/.qlib/qlib_data/cn_data')}


qlib: ['c:\\Users\\shiva\\AppData\\Local\\Programs\\Python\\Python38\\lib\\site-packages\\qlib']


## Init Qlib Data Handler (Data handler is the configuration to grab data)

In [4]:
from qlib.contrib.data.handler import Alpha158,Alpha360
handler_kwargs = {
    "start_time": "2016-01-01",
    "end_time": "2020-01-01",
    "fit_start_time": "2016-01-01",
    "fit_end_time": "2020-01-01",
    "instruments": MARKET,
    "infer_processors": [
        {"class": "RobustZScoreNorm",
          "kwargs":{
              "fields_group": "feature",
              "clip_outlier": True
            }
        },
        {"class": "Fillna",
          "kwargs":{
              "fields_group": "feature"}}],
    "learn_processors": [{
        "class": "DropnaLabel"},
        {"class": "CSRankNorm",
          "kwargs":{
              "fields_group": "label"}}
    ],
    "label":["Ref($close, -2) / Ref($close, -1) - 1"]
}
hd = Alpha360(**handler_kwargs)
dataset_conf = {
    "class": "DatasetH",
    "module_path": "qlib.data.dataset",
    "kwargs": {
        "handler": hd,
        "segments": {
            "train": ("2016-01-01", "2018-6-30"),
            "valid": ("2018-07-01", "2018-12-31"),
            "test": ("2019-01-01", "2020-01-01"),
        },
    },
}

KeyboardInterrupt: 

## Show how data handler grab data

In [7]:
from qlib.utils import init_instance_by_config
dataset_= init_instance_by_config(dataset_conf)
df = dataset_.prepare("train",col_set=["feature","label"])
df

feature                                          \
                        CLOSE59   CLOSE58   CLOSE57   CLOSE56   CLOSE55   
datetime   instrument                                                     
2016-01-04 SH600000   -1.126194 -1.008027 -0.880314 -0.867696 -0.595524   
           SH600008   -0.184281 -0.091406 -0.171548 -0.424415 -0.419211   
           SH600009   -0.108012 -0.125249 -0.109665 -0.255521 -0.245955   
           SH600010    0.989734  0.753716  0.808222  0.519128  0.572980   
           SH600011    0.455407  0.247022  0.313659  0.263040  0.276873   
...                         ...       ...       ...       ...       ...   
2018-06-29 SZ300136    1.303131  1.039014  0.851422  0.933580  1.334124   
           SZ300144   -0.949527 -1.109916 -1.054526 -0.972724 -1.113224   
           SZ300251    1.607630  1.434895  1.194196  1.098719  1.003325   
           SZ300408   -0.071530 -0.313738 -0.225318 -0.261688 -0.039709   
           SZ300433    1.267888  1.009930  0.870191  0.740745 -0.088189   

                                                                         ...  \
                        CLOSE54   CLOSE53   CLOSE52   CLOSE51   CLOSE50  ...   
datetime   instrument                                                    ...   
2016-01-04 SH600000   -0.927388 -0.923720 -0.861172 -0.912752 -0.950446  ...   
           SH600008   -0.141428  0.009386  0.401560  0.474637  0.229503  ...   
           SH600009    0.002618 -0.017777 -0.015897  0.209532  0.015895  ...   
           SH600010    0.789549  0.942236  1.336943  1.377439  2.198421  ...   
           SH600011    0.566829  0.621679  0.895285  0.896344  0.780490  ...   
...                         ...       ...       ...       ...       ...  ...   
2018-06-29 SZ300136    1.407368  1.693937  1.525842  1.146055  1.082973  ...   
           SZ300144   -1.140714 -1.191843 -1.281620 -1.325351 -1.403060  ...   
           SZ300251    0.991498  0.751054  0.904031  0.937665  0.955617  ...   
           SZ300408    0.030784  0.104240  0.034967 -0.058630 -0.360067  ...   
           SZ300433    0.307420  0.477986  0.436689  0.332363  0.086461  ...   

                                                                         \
                        VOLUME8   VOLUME7   VOLUME6   VOLUME5   VOLUME4   
datetime   instrument                                                     
2016-01-04 SH600000    0.385022  0.940287  0.580990  0.520288  0.289184   
           SH600008    0.715785  2.191809  0.458130  0.112290  1.227424   
           SH600009    1.601055  2.662354 -0.504095 -0.494579  1.158635   
           SH600010    1.773483  0.720130  0.027917 -0.605187  0.097492   
           SH600011    0.531823  1.296508  0.174132 -0.475570  0.242690   
...                         ...       ...       ...       ...       ...   
2018-06-29 SZ300136    0.558303  1.109834  0.448312 -0.072796 -0.242319   
           SZ300144   -0.314124 -0.663355 -0.668377 -0.901623 -1.156276   
           SZ300251   -0.141182 -0.636418 -0.786099 -0.286050 -0.736194   
           SZ300408    0.852179 -0.102631 -0.235150 -0.428136 -0.607546   
           SZ300433    0.384459  0.359060  0.764071 -0.083671 -0.246875   

                                                             \
                        VOLUME3   VOLUME2   VOLUME1 VOLUME0   
datetime   instrument                                         
2016-01-04 SH600000    0.177203 -0.383985 -0.890843     0.0   
           SH600008    0.002929  0.509405 -0.215640     0.0   
           SH600009   -0.705676 -0.102309 -0.324185     0.0   
           SH600010   -0.479283 -0.432742 -0.730405     0.0   
           SH600011   -0.502024 -0.426651 -0.754623     0.0   
...                         ...       ...       ...     ...   
2018-06-29 SZ300136   -0.470804 -0.868333 -0.664394     0.0   
           SZ300144   -0.961849 -1.026384  0.911242     0.0   
           SZ300251   -0.697346 -1.176048 -0.971235     0.0   
           SZ300408    0.075141 -0.680968 -0.

## Train Model 

In [8]:
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord, SigAnaRecord
from qlib.utils import init_instance_by_config
import torch
model = init_instance_by_config(
    {
        "class": "GRU",
        "module_path": "qlib.contrib.model.pytorch_gru",
        "kwargs": {
            "d_feat": 6,
            "hidden_size": 64,
            "num_layers": 2,
            "dropout": 0.0,
            "n_epochs": 200,
            "lr": 1e-3,
            "early_stop": 20,
            "loss": "mse",
            "GPU":4
        },
    }
)
dataset1 = init_instance_by_config(dataset_conf)

# start exp to train model
with R.start(experiment_name=EXP_NAME):
    model.fit(dataset1)
    R.save_objects(trained_model=model)

    rec = R.get_recorder()
    rid = rec.id  # save the record id

    # Inference and saving signal
    sr = SignalRecord(model, dataset1, rec)
    sr.generate()

[64517:MainThread](2023-11-23 22:15:40,405) INFO - qlib.GRU - [pytorch_gru.py:59] - GRU pytorch version...
[64517:MainThread](2023-11-23 22:15:40,409) INFO - qlib.GRU - [pytorch_gru.py:105] - GRU parameters setting:
d_feat : 6
hidden_size : 64
num_layers : 2
dropout : 0.0
n_epochs : 200
lr : 0.001
metric : 
batch_size : 2000
early_stop : 20
optimizer : adam
loss_type : mse
visible_GPU : 4
use_GPU : True
seed : None
[64517:MainThread](2023-11-23 22:15:40,413) INFO - qlib.GRU - [pytorch_gru.py:119] - model:
GRUModel(
  (rnn): GRU(6, 64, num_layers=2, batch_first=True)
  (fc_out): Linear(in_features=64, out_features=1, bias=True)
)
[64517:MainThread](2023-11-23 22:15:40,415) INFO - qlib.GRU - [pytorch_gru.py:120] - model size: 0.0370 MB
[64517:MainThread](2023-11-23 22:15:40,448) INFO - qlib.workflow - [exp.py:258] - Experiment 466892454790696299 starts running ...
[64517:MainThread](2023-11-23 22:15:40,463) INFO - qlib.workflow - [recorder.py:341] - Recorder ce76a134c0304825b1877606924c2

[64517:MainThread](2023-11-23 22:15:51,311) INFO - qlib.GRU - [pytorch_gru.py:241] - training...
[64517:MainThread](2023-11-23 22:15:51,314) INFO - qlib.GRU - [pytorch_gru.py:245] - Epoch0:
[64517:MainThread](2023-11-23 22:15:51,316) INFO - qlib.GRU - [pytorch_gru.py:246] - training...
[64517:MainThread](2023-11-23 22:15:55,043) INFO - qlib.GRU - [pytorch_gru.py:248] - evaluating...
[64517:MainThread](2023-11-23 22:15:56,029) INFO - qlib.GRU - [pytorch_gru.py:251] - train -0.994643, valid -0.996446
[64517:MainThread](2023-11-23 22:15:56,034) INFO - qlib.GRU - [pytorch_gru.py:245] - Epoch1:
[64517:MainThread](2023-11-23 22:15:56,036) INFO - qlib.GRU - [pytorch_gru.py:246] - training...
[64517:MainThread](2023-11-23 22:15:59,177) INFO - qlib.GRU - [pytorch_gru.py:248] - evaluating...
[64517:MainThread](2023-11-23 22:16:01,967) INFO - qlib.GRU - [pytorch_gru.py:251] - train -0.993642, valid -0.996661
[64517:MainThread](2023-11-23 22:16:01,970) INFO - qlib.GRU - [pytorch_gru.py:245] - Epoc

in record gen pred= datetime    instrument
2019-01-02  SH600000     -0.034272
            SH600004      0.051557
            SH600009     -0.102355
            SH600010      0.065659
            SH600011     -0.120147
                            ...   
2019-12-31  SZ300347      0.048555
            SZ300408      0.054300
            SZ300413     -0.165213
            SZ300433      0.013241
            SZ300498      0.008121
Length: 73200, dtype: float32
'The following are prediction results of the GRU model.'
                          score
datetime   instrument          
2019-01-02 SH600000   -0.034272
           SH600004    0.051557
           SH600009   -0.102355
           SH600010    0.065659
           SH600011   -0.120147


[64517:MainThread](2023-11-23 22:18:21,921) INFO - qlib.timer - [log.py:128] - Time cost: 0.000s | waiting `async_log` Done
